 # Análise de Inadimplência - X-Health

 ## Índice da Análise

 1. Deployment Phase

     * API development

         * Prediction Class

         * API Handler

         * API Tester

 # Setup Phase

 ## Libraries

In [1]:
try:
    # Toolbox libraries
    import pickle
    import json
    import pandas as pd
    import numpy as np
    import inflection
    print("Import successful")
    
except Exception as e:
    
    print("Error while importing libraries: ", "\n", e)



Import successful


#### Predict.py

In [2]:
def open_model(model_name, folder = 'Dev/'):
    
    """
    Load a saved model from disk.
    
    Parameters
    ----------
    model_name : str
        Name of the model to load.
        
    Returns
    -------
    model : object
        The loaded model, or None if loading failed.
    """
    
    file_name = model_name
    model_name = './Models/' + folder + model_name + '.pkl'
    
    try:
        
        model = pickle.load(open(model_name, 'rb'))
        print(f'{file_name} loaded successfully!')
        
        return model
    
    except Exception as e:
        
        print(f'Error loading model {file_name}: {str(e)}')
        
        return None


def make_prediction(model, data):
    """Realiza a predição usando o modelo scikit-learn"""
    try:
        # Fazer a predição - usando a interface scikit-learn
        prediction_proba = model.predict_proba(data)
        
        # Obter a classe predita (0 ou 1)
        prediction = model.predict(data)
        
        # Retornar o resultado da primeira amostra
        result = int(prediction[0])
        
        return result
    
    except Exception as e:
        
        raise ValueError(f"Erro ao fazer a predição: {e}")

Api.py

In [3]:
def validate_json(data):
    
    """Verifica se os campos necessários estão presentes no JSON"""
    
    required_fields = ['default_3months', 'ioi_36months', 'ioi_3months', 'valor_por_vencer', 'valor_quitado', 'valor_total_pedido']
    
    for field in required_fields:
        
        if field not in data:
            
            return False, f"Campo obrigatório ausente: {field}"
    
    return True, "JSON válido"


def process_json_for_prediction(data):
    """Processa o JSON para o formato adequado para predição"""
    try:
        
        df_raw = pd.DataFrame([data])
        
        snakecase = lambda column: inflection.underscore(column)
        Colunas_new = list(map(snakecase, df_raw.columns))
        df_raw.columns = Colunas_new
        
        df_raw = df_raw.replace("missing", np.nan)
        
        # valor_vencido, valor_quitado, ioi_3months, ioi_36months
        df_raw['var_ioi'] = df_raw[['ioi_3months', 'ioi_36months']].std(axis=1)
        df_raw['taxa_cresc_quitado'] = df_raw['valor_quitado'] / (df_raw['ioi_3months'] + 1)
        df_raw.columns.name = None
        df_transformed = df_raw.copy()
        # 1. default_3months, ioi_36months, ioi_3months, valor_por_vencer, valor_quitado, valor_total_pedido
        df_transformed['default_3months_transformed'] = np.log1p(df_raw['default_3months'])
        df_transformed['default_3months_scaled'] = (df_transformed['default_3months_transformed'] - df_transformed['default_3months_transformed'].min()) / \
                                                    (df_transformed['default_3months_transformed'].max() - df_transformed['default_3months_transformed'].min())
        # 2. ioi_36months
        df_transformed['ioi_36months_transformed'] = np.log(df_raw['ioi_36months'].replace(0, 0.1))
        df_transformed['ioi_36months_scaled'] = (df_transformed['ioi_36months_transformed'] - df_transformed['ioi_36months_transformed'].min()) / \
                                                (df_transformed['ioi_36months_transformed'].max() - df_transformed['ioi_36months_transformed'].min())
        # 3. ioi_3months
        df_transformed['ioi_3months_transformed'] = np.log(df_raw['ioi_3months'].replace(0, 0.1))
        df_transformed['ioi_3months_scaled'] = (df_transformed['ioi_3months_transformed'] - df_transformed['ioi_3months_transformed'].min()) / \
                                                (df_transformed['ioi_3months_transformed'].max() - df_transformed['ioi_3months_transformed'].min())
        # 4. valor_por_vencer
        df_transformed['valor_por_vencer_transformed'] = np.log1p(df_raw['valor_por_vencer'])
        df_transformed['valor_por_vencer_scaled'] = (df_transformed['valor_por_vencer_transformed'] - df_transformed['valor_por_vencer_transformed'].min()) / \
                                                    (df_transformed['valor_por_vencer_transformed'].max() - df_transformed['valor_por_vencer_transformed'].min())
        # 6. valor_quitado
        df_transformed['valor_quitado_transformed'] = np.log1p(df_raw['valor_quitado'])
        df_transformed['valor_quitado_scaled'] = (df_transformed['valor_quitado_transformed'] - df_transformed['valor_quitado_transformed'].min()) / \
                                                (df_transformed['valor_quitado_transformed'].max() - df_transformed['valor_quitado_transformed'].min())
        # 15. valor_total_pedido
        df_transformed['valor_total_pedido_transformed'] = np.log1p(np.abs(df_raw['valor_total_pedido'])) * np.sign(df_raw['valor_total_pedido'])
        df_transformed['valor_total_pedido_scaled'] = (df_transformed['valor_total_pedido_transformed'] - df_transformed['valor_total_pedido_transformed'].min()) / \
                                                    (df_transformed['valor_total_pedido_transformed'].max() - df_transformed['valor_total_pedido_transformed'].min())
        # 19. var_ioi
        df_transformed['var_ioi_transformed'] = np.log1p(df_raw['var_ioi'])
        df_transformed['var_ioi_scaled'] = (df_transformed['var_ioi_transformed'] - df_transformed['var_ioi_transformed'].min()) / \
                                            (df_transformed['var_ioi_transformed'].max() - df_transformed['var_ioi_transformed'].min())
        # 20. taxa_cresc_quitado
        df_transformed['taxa_cresc_quitado_transformed'] = np.log1p(df_raw['taxa_cresc_quitado'])
        df_transformed['taxa_cresc_quitado_scaled'] = (df_transformed['taxa_cresc_quitado_transformed'] - df_transformed['taxa_cresc_quitado_transformed'].min()) / \
                                                    (df_transformed['taxa_cresc_quitado_transformed'].max() - df_transformed['taxa_cresc_quitado_transformed'].min())
        
        # Substituir infinitos por 0 (caso ocorram em alguma divisão)
        df_transformed = df_transformed.replace([np.inf, -np.inf], 0)
        
        # Ausência de NaN após transformações
        
        if df_transformed.isna().sum().sum() > 0:
            
            df_transformed = df_transformed.fillna(0)
        
        columns_to_keep = [col for col in df_transformed.columns if '_scaled' in col]
        
        df_final = df_transformed[columns_to_keep]
        
        df_final = df_final[['default_3months_scaled', 'ioi_36months_scaled', 'ioi_3months_scaled', 'valor_por_vencer_scaled', 
                                'valor_quitado_scaled', 'valor_total_pedido_scaled', 'var_ioi_scaled', 'taxa_cresc_quitado_scaled']]
                                
        return df_final, "Dados processados com sucesso"
    
    except Exception as e:
        
        return None, f"Erro ao processar dados: {e}"

Exemplo de Input para predição do Modelo:

{"default_3months": 0.0,

"ioi_36months": 22.961318993,

"ioi_3months": 9.7398055882,

"valor_por_vencer": 6993.3971923151,

"valor_quitado": 92981.1128787256,

"valor_total_pedido": 4545.5232294998}

In [4]:
json_path = './Test/deploy_test.json'

In [5]:
with open (json_path, 'r') as json_file:
    json_data = json.load(json_file)

In [6]:
if json_data is not None:
    
    is_valid, validation_msg = validate_json(json_data)
    
    if is_valid:
        
        processed_data, process_msg = process_json_for_prediction(json_data)
        
        if processed_data is not None:
            
            model = open_model("XGBoost_Best_Model_Prod", folder = 'Prod/')
            
            if model is not None:
                
                try: 
                    
                    prediction_result = make_prediction(model, processed_data)
                    
                except:
                    
                    print(f"Erro ao fazer predição: {e}")
                    
                if prediction_result is not None:
                    
                    if prediction_result == 1:
                        
                        print("✅ O Cliente é um Bom Pagador")
                        
                    else:
                        
                        print("❌ O Cliente é um Provável Inadimplente")
                        
            else:
                
                print("Não foi possível carregar o modelo")
                
        else:
            
            print(process_msg)
            
    else:
        
        print(validation_msg)
        
else:
    print("Por favor, carregue um arquivo JSON antes de analisar")

XGBoost_Best_Model_Prod loaded successfully!
❌ O Cliente é um Provável Inadimplente
